In [35]:
# !pip install tqdm matplotlib numpy captum scikit-learn pretrainedmodels

In [4]:
# !pip install kaggle
# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [5]:
# # !pip install zipfile36
# with zipfile.ZipFile("chest-xray-pneumonia.zip","r") as zip_ref:
#     zip_ref.extractall("chest_xray")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


from tqdm import tqdm

In [4]:
data_dir = './xray_dataset/'
# convert all input images to square tensors
# this step is probably ruining the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    # transforms.ColorJitter(brightness=0.4, contrast=0.4),
    # transforms.ColorJitter(brightness=0.3),
    # transforms.RandomRotation(15),
    # normalize the data
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Normalize((0.5,), (0.5,)),
])


In [5]:
# Load training data
train_data = ImageFolder(root=data_dir + 'train', transform=transform)

# Load validation data
val_data = ImageFolder(root=data_dir + 'val', transform=transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True, num_workers=8)

# Load test data
test_data = ImageFolder(root=data_dir + 'test', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=8)

In [10]:
from torch.utils.data import DataLoader, WeightedRandomSampler

# Calculate class weights
class_counts = [0] * len(train_data.classes)
print(class_counts)
labels_ = [label for _, label in tqdm(train_data)]
for label in tqdm(labels_):
    class_counts[label] += 1
class_weights = [1.0 / count for count in class_counts]
sample_weights = [class_weights[label] for label in labels_]


# Create weighted sampler and data loader
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
# train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

[0, 0]


100%|██████████| 5216/5216 [00:00<00:00, 3052105.14it/s]


In [11]:
import random
import torchvision.datasets as datasets
from torch.utils.data import Dataset

class RandomLabelDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, index):
        img, label = self.dataset[index]
        rand_label = random.randint(0, len(self.dataset.classes) - 1)
        return img, rand_label

# create the random label dataset
random_label_data = RandomLabelDataset(train_data)
train_loader = DataLoader(random_label_data, batch_size=32, sampler=sampler)

### Data loaders

In [12]:
# from torch.utils.data import DataLoader, TensorDataset

# # Create TensorDataset objects
# train_dataset = TensorDataset(train_data, train_labels)
# val_dataset = TensorDataset(val_data, val_labels)
# test_dataset = TensorDataset(test_data, test_labels)

# # Create DataLoader objects
batch_size = 32
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### Let's do some displaying

### Let's do some training

In [13]:
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.nn.functional as F
import logging

logging.basicConfig(filename='logs.log', level=logging.INFO)

import torchvision.models as models


model = models.resnet18(pretrained=True)

# Modify first layer to accept 3-channel input (RGB)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify last linear layer to output 2 classes
model.fc = nn.Sequential(
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.2),
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(),
    nn.BatchNorm1d(256, eps=1e-05, momentum=0.1),
    nn.Dropout(p=0.2),
    nn.Linear(in_features=256, out_features=64),
    nn.ReLU(),
    nn.BatchNorm1d(64, eps=1e-05, momentum=0.1),
    nn.Dropout(p=0.2),
    nn.Linear(in_features=64, out_features=2),
)

# Use AdaptiveAvgPool2d to allow for variable input image sizes
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))


device = 'cuda'
net = model.to(device)


criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device))


optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [14]:
EPOCHS = 20
PRINT_EVERY = 1000

for epoch in range(EPOCHS):
      
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_loader), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # move inputs and labels to MPS device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Calculate test set loss and accuracy
    test_loss = 0.0
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        for test_data in test_loader:
            test_inputs, test_labels = test_data
            test_inputs = test_inputs.to(device)
            test_labels = test_labels.to(device)
            test_outputs = net(test_inputs) sla
            test_loss += criterion(test_outputs, test_labels).item()
            _, test_predicted = torch.max(test_outputs.data, 1)
            test_total += test_labels.size(0)
            test_correct += (test_predicted == test_labels).sum().item()
    test_loss /= len(test_loader)
    test_acc = 100 * test_correct / test_total

    msg = f'Test set loss for EPOCH {epoch}: {test_loss:.3f}, test_acc: {test_acc:.2f}%'
    print(msg)
    logging.info(msg)
    
    scheduler.step()


print('Finished Training')

100%|██████████| 163/163 [01:52<00:00,  1.45it/s]


Test set loss for EPOCH 0: 0.686, test_acc: 41.03%


100%|██████████| 163/163 [01:56<00:00,  1.40it/s]


Test set loss for EPOCH 1: 0.689, test_acc: 37.02%


100%|██████████| 163/163 [01:55<00:00,  1.41it/s]


...

Test set loss for EPOCH 8: 0.694, test_acc: 37.82%


100%|██████████| 163/163 [01:52<00:00,  1.45it/s]


Test set loss for EPOCH 9: 0.691, test_acc: 37.50%


 42%|████▏     | 69/163 [00:48<01:05,  1.43it/s]


KeyboardInterrupt: 

In [15]:
# # save the model
MODEL_PATH = './xray_net_randomized.pth'
torch.save(net.state_dict(), MODEL_PATH)


In [16]:
print(f'Test set loss: {test_loss:.3f}, test_acc: {test_acc:.2f}%')

Test set loss: 0.691, test_acc: 37.50%
